In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import DataLoader, TensorDataset

In [2]:
df = pd.read_csv('/kaggle/input/greetings/greetings.csv')

In [3]:
df

,Question,Label
0,"If we take up a 6 months internship, like othe...",No
1,Can we do a 6 month internship without droppin...,No
2,Can we do internship and in parallel register ...,No
3,Are external BTP/IP collaborations allowed?,No
4,Can a B.Tech. student do BTP with a guest or a...,No
...,...,...
223,"Hey, how’s everything on your end?",Yes
224,"Hello, hope today’s been kind to you.",Yes
225,"Hi, how are things going lately?",Yes
226,What’s been keeping you busy?,Yes


In [4]:
print(df['Label'].isnull().sum())  # Check for NaNs

0


In [5]:
df['Label'] = df['Label'].map({'Yes': 1, 'No': 0})

# Text vectorization using TF-IDF
vectorizer = TfidfVectorizer(max_features=50)
X_tfidf = vectorizer.fit_transform(df['Question']).toarray()

# Convert to 3D tensor (batch_size, seq_length, feature_size)
X_data = np.expand_dims(X_tfidf, axis=1)  # seq_length=1

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_data, dtype=torch.float32)
y_tensor = torch.tensor(df['Label'].values, dtype=torch.long)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2,shuffle = True)


In [6]:
print(y_tensor.unique())

tensor([0, 1])


In [7]:
# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [8]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 64).to(x.device)
        c0 = torch.zeros(2, x.size(0), 64).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Model parameters
input_size = 50  # Same as max_features in TF-IDF
hidden_size = 64
num_layers = 2
output_size = 2


In [9]:
# Instantiate model, loss, and optimizer
model = LSTMClassifier(input_size, hidden_size, num_layers, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        print(batch_y)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

tensor([0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1])
tensor([1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1])
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1])
tensor([1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0])
tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1])
tensor([1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1])
tensor([1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0])
tensor([1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1])
tensor([1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0])
tensor([0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0])
tensor([0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0])
tensor([0, 0, 0, 1, 1, 1])
Epoch [1/10], Loss: 0.6928876042366028
tensor([1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1])
tensor([0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0])
tensor([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0])
tensor([1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0])
tensor([0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1])
tensor([1, 1, 1, 1, 1,

In [10]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    predicted_labels = torch.argmax(test_outputs, dim=1)
    accuracy = (predicted_labels == y_test).sum().item() / y_test.size(0)
    print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 97.83%


In [11]:
torch.save(model.state_dict(), "lstm_text_classifier.pth")
print("Model saved successfully!")

Model saved successfully!


In [12]:
model_loaded = LSTMClassifier(input_size, hidden_size, num_layers, output_size)
model_loaded.load_state_dict(torch.load("/kaggle/working/lstm_text_classifier.pth"))
model_loaded.eval()  # Set model to evaluation mode

<ipython-input-12-9425928f85f6>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_loaded.load_state_dict(torch.load("/kaggle/working/lstm_text_classifier.pth"))


LSTMClassifier(
  (lstm): LSTM(50, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=2, bias=True)
)

In [24]:
def predict(text, model, vectorizer):
    model.eval()  # Set to evaluation mode

    # Convert text to TF-IDF vector
    text_vector = vectorizer.transform([text]).toarray()
    text_tensor = torch.tensor(np.expand_dims(text_vector, axis=1), dtype=torch.float32)

    # Get model prediction
    with torch.no_grad():
        output = model(text_tensor)
        predicted_label = torch.argmax(output, dim=1).item()

    return "yes" if predicted_label == 1 else "no"

# Example usage
new_text = "Hello, How are you?"
predicted_label = predict(new_text, model_loaded, vectorizer)
print(f"Predicted Label: {predicted_label}")

Predicted Label: yes
